# Load Processed Data 

In [1]:
%run ./Airbnb_DataSource_n_DataWrangling.ipynb

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


total 3162
-rw-r--r-- 1 khyat 197609 3124181 Sep 26 15:50 Airbnb.pptx
drwxr-xr-x 1 khyat 197609       0 Oct  1 13:05 Data
drwxr-xr-x 1 khyat 197609       0 Oct  1 12:22 Data_Original
drwxr-xr-x 1 khyat 197609       0 Oct  3 01:37 Data_Output
drwxr-xr-x 1 khyat 197609       0 Oct  1 12:55 Temp
drwxr-xr-x 1 khyat 197609       0 Oct  2 23:55 Visualization_Output
-rw-r--r-- 1 khyat 197609      39 May  9 10:15 apikey.txt
-rw-r--r-- 1 khyat 197609   94729 Oct  3 01:56 treeClassifier.txt
-rw-r--r-- 1 khyat 197609     165 Sep 26 13:19 ~$Airbnb.pptx


C:\Users\khyat\.conda\envs\R_Work\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (61,62,94,95) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
./Airbnb_DataSource_n_DataWrangling.ipynb:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  "cells": [


In [6]:
df.head(n=2)
# print(df.shape)

,id,scrape_id,last_scraped,name,summary,description,experiences_offered,host_id,host_name,host_since,...,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,price_group,room_type_num
0,13913,20190710134553,2019-07-11,Holiday London DB Room Let-on going,My bright double bedroom with a large window h...,My bright double bedroom with a large window h...,business,54730,Alina,2009-11-16,...,moderate,f,f,3,1,2,0,0.15,medium,0
1,15400,20190710134553,2019-07-11,Bright Chelsea Apartment. Chelsea!,Lots of windows and light. St Luke's Gardens ...,Lots of windows and light. St Luke's Gardens ...,romantic,60302,Philippa,2009-12-05,...,strict_14_with_grace_period,t,t,1,1,0,0,0.73,medium,1


# Predictors of Price to be a Successful Host

For host to understand what factors are important to decide the best price of the room

In [61]:
predictors_set = ['minimum_nights','number_of_reviews','calculated_host_listings_count','availability_365','room_type_num']
x = df.loc[:, predictors_set]
y = df.price_group

In [31]:
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.25,train_size=0.75)

In [32]:
train_accuracy=[]
test_accuracy=[]
depth_range=range(2,15)
for depth in depth_range:
    tree=DecisionTreeClassifier(max_depth = depth)
    tree.fit(train_x, train_y)
    train_accuracy.append(np.mean(train_y == tree.predict(train_x)))
    test_accuracy.append(np.mean(test_y == tree.predict(test_x)))

In [34]:
print(train_accuracy)
print(test_accuracy)

[0.6201017000352426, 0.6468189370164633, 0.65027606692735, 0.6511487404970883, 0.6539178008626043, 0.6576098813499589, 0.6630640911608237, 0.6668736469364123, 0.6724453320355112, 0.677513551613607, 0.6842767717790793, 0.6935405373655328, 0.7029049960561867]
[0.6180838745406031, 0.6413935457886523, 0.6439611337662992, 0.6448673412878215, 0.6453204450485828, 0.6483411367869909, 0.6519156220107738, 0.6514625182500126, 0.6554397623722499, 0.6528218295322963, 0.6530735538438303, 0.6473845844031617, 0.6485928610985249]


In [38]:
test_acc_score = pd.DataFrame(test_accuracy, index=depth_range, columns=['score'])
test_acc_score = test_acc_score.sort_values(by="score",ascending=False)
test_acc_score.head(n = 4)

,score
10,0.655440
12,0.653074
11,0.652822
8,0.651916


In [41]:
from sklearn import tree
# Fitting model with best hyper parameters
DecisionTree_m1 = tree.DecisionTreeClassifier(max_depth=10)
DecisionTree_m1.fit(train_x, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=10,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [53]:
# Compare y_test and predicted y
m1_predictions = DecisionTree_m1.predict(test_x)
pred = test_y.to_frame() # convert to dataframe
pred['prediction'] = m1_predictions
# pred.head(n=5)

In [58]:
# score of test
dt_m1_score = DecisionTree_m1.score(test_x, test_y)
dt_m1_score = [dt_m1_score]
dt_m1_score_df = pd.DataFrame(dt_m1_score,columns=['score'],dtype=float)
dt_m1_score_df

,score
0,0.65549


In [66]:
# Calculate most important features
feature_imp = DecisionTree_m1.feature_importances_
feature_imp_df = pd.DataFrame({"Feature":predictors_set,"Importance":feature_imp})
feature_imp_df = feature_imp_df.sort_values(by="Importance",ascending=False)
feature_imp_df

,Feature,Importance
4,room_type_num,0.763086
2,calculated_host_listings_count,0.106894
3,availability_365,0.064453
1,number_of_reviews,0.038475
0,minimum_nights,0.027093


In [64]:
# save the model to disk
m1_path = path + '\\Data_Output\\' + "DecisionTree_m1.sav"
pickle.dump(DecisionTree_m1, open(m1_path,'wb'))

In [67]:
# export the results as csv file
test_acc_score.to_csv(path + '\\Data_Output\\' + 'depth_score.csv')
pred.to_csv(path + '\\Data_Output\\' + 'predictions.csv')
dt_m1_score_df.to_csv(path + '\\Data_Output\\' + 'DecisionTree_m1_score.csv')
feature_imp_df.to_csv(path + '\\Data_Output\\' + 'feature_importance.csv')

In [119]:
import os
os.environ["PATH"] += os.pathsep + 'C:\Program Files (x86)\Graphviz2.38\bin'

In [112]:
# from graphviz import Source
dot_data = tree.export_graphviz(DecisionTree_m1, out_file=None,
                             feature_names=predictors_set,
                             class_names=['low', 'medium', 'high'],
                             filled=True, rounded=True,
                             special_characters=True)

# graph = graphviz.Source(dot_data, format = "png")
# graph.render("tree_graph.png")

In [113]:
(graph, ) = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph[0].write_pdf("iris.pdf")


AttributeError: 'str' object has no attribute 'getvalue'

In [114]:

with open("treeClassifier.txt", "w") as f:
 f = tree.export_graphviz(DecisionTree_m1, out_file=f)


In [122]:
! sudo apt-get install graphviz

'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [123]:
dot_data = tree.export_graphviz(DecisionTree_m1, class_names=['low', 'medium', 'high'], out_file=None)
graph = graphviz.Source(dot_data)
graph

ExecutableNotFound: failed to execute ['dot', '-Tsvg'], make sure the Graphviz executables are on your systems' PATH